![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Clinical Named Entity Recognition Models

Please make sure that your cluster is setup properly according to https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks

**NER Model Implementation in Spark NLP**

  The deep neural network architecture for NER model in
Spark NLP is BiLSTM-CNN-Char framework. a slightly modified version of the architecture proposed by Jason PC Chiu and Eric Nichols ([Named Entity Recognition with Bidirectional LSTM-CNNs
](https://arxiv.org/abs/1511.08308)). It is a neural network architecture that
automatically detects word and character-level features using a
hybrid bidirectional LSTM and CNN architecture, eliminating
the need for most feature engineering steps.

  In the original framework, the CNN extracts a fixed length
feature vector from character-level features. For each word,
these vectors are concatenated and fed to the BLSTM network
and then to the output layers. They employed a stacked
bi-directional recurrent neural network with long short-term
memory units to transform word features into named entity
tag scores. The extracted features of each word are fed into a
forward LSTM network and a backward LSTM network. The
output of each network at each time step is decoded by a linear
layer and a log-softmax layer into log-probabilities for each tag
category. These two vectors are then simply added together to
produce the final output. In the architecture of the proposed framework in the original paper, 50-dimensional pretrained word
embeddings is used for word features, 25-dimension character
embeddings is used for char features, and capitalization features
(allCaps, upperInitial, lowercase, mixedCaps, noinfo) are used
for case features.

## Blogposts and videos:

- [How to Setup Spark NLP for HEALTHCARE on UBUNTU - Video](https://www.youtube.com/watch?v=yKnF-_oz0GE)

- [Named Entity Recognition (NER) with BERT in Spark NLP](https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77)

- [State of the art Clinical Named Entity Recognition in Spark NLP - Youtube](https://www.youtube.com/watch?v=YM-e4eOiQ34)

- [Named Entity Recognition for Healthcare with SparkNLP NerDL and NerCRF](https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571)

- [Named Entity Recognition for Clinical Text](https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.2
sparknlp_jsl.version : 3.5.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.8:7077 
 AppName 
 Databricks Shell

# Clinical NER Pipeline (with pretrained models)

## Clinical NER Models

- **Clinical NER Model List**

|   index | model_name                     |   index | model_name                        |   index | model_name                        |   index | model_name                 |
|--------:|:-------------------------------|--------:|:----------------------------------|--------:|:----------------------------------|--------:|:---------------------------|
|       1 | jsl_ner_wip_clinical           |       2 | ner_chexpert                      |       3 | ner_deid_subentity (German)       |       4 | ner_jsl_greedy             |
|       5 | jsl_ner_wip_greedy_clinical    |       6 | ner_clinical                      |       7 | ner_diseases_large                |       8 | ner_jsl_slim               |
|       9 | jsl_ner_wip_modifier_clinical  |      10 | ner_clinical_icdem                |      11 | ner_drugs                         |      12 | ner_measurements_clinical  |
|      13 | jsl_rd_ner_wip_greedy_clinical |      14 | ner_clinical_large                |      15 | ner_drugs_greedy                  |      16 | ner_medmentions_coarse     |
|      17 | ner_ade_clinical               |      18 | ner_clinical_large_en             |      19 | ner_drugs_large                   |      20 | ner_posology               |
|      21 | ner_ade_clinicalbert           |      22 | ner_deid_augmented                |      23 | ner_events_admission_clinical     |      24 | ner_posology_experimental  |
|      25 | ner_ade_healthcare             |      26 | ner_deid_enriched                 |      27 | ner_events_clinical               |      28 | ner_posology_greedy        |
|      29 | ner_anatomy                    |      30 | ner_deid_generic_augmented        |      31 | ner_events_healthcare             |      32 | ner_posology_healthcare    |
|      33 | ner_anatomy_coarse             |      34 | ner_deid_generic (German)         |      35 | ner_genetic_variants              |      36 | ner_posology_large         |
|      37 | ner_bacterial_species          |      38 | ner_deid_large                    |      39 | ner_healthcare                    |      40 | ner_posology_small         |
|      41 | ner_bionlp                     |      42 | ner_deid_sd                       |      43 | ner_human_phenotype_gene_clinical |      44 | ner_profiling_clinical     |
|      45 | ner_cancer_genetics            |      46 | ner_deid_sd_large                 |      47 | ner_human_phenotype_go_clinical   |      48 | ner_radiology              |
|      49 | ner_cellular                   |      50 | ner_deid_subentity_augmented      |      51 | ner_jsl                           |      52 | ner_radiology_wip_clinical |
|      53 | ner_chemicals                  |      54 | ner_deid_synthetic                |      55 | ner_jsl_enriched                  |      56 | ner_risk_factors           |
|      57 | ner_chemprot_clinical          |      58 | ner_deidentify_dl                 |      59 | ner_nihss                         |      60 | ner_biomarker              |
|      61 | ner_abbreviation_clinical      |      62 | ner_deid_subentity_augmented_i2b2 |      63 | ner_diseases                      |      64 | ner_drugprot_clinical      |
|      65 | ner_nature_nero_clinical_en    |      66 | ner_supplement_clinical_en        |         |                                   |         |                            |




- **BioBert NER Models**

|   index | model_name                    |   index | model_name                |   index | model_name                       |   index | model_name                 |
|--------:|:------------------------------|--------:|:--------------------------|--------:|:---------------------------------|--------:|:---------------------------|
|       1 | jsl_ner_wip_greedy_biobert    |       7 | ner_cellular_biobert      |      13 | ner_events_biobert               |      18 | ner_jsl_greedy_biobert     |
|       2 | jsl_rd_ner_wip_greedy_biobert |       8 | ner_chemprot_biobert      |      14 | ner_human_phenotype_gene_biobert |      19 | ner_posology_biobert       |
|       3 | ner_ade_biobert               |       9 | ner_clinical_biobert      |      15 | ner_human_phenotype_go_biobert   |      20 | ner_posology_large_biobert |
|       4 | ner_anatomy_biobert           |      10 | ner_deid_biobert          |      16 | ner_jsl_biobert                  |      21 | ner_profiling_biobert      |
|       5 | ner_anatomy_coarse_biobert    |      11 | ner_deid_enriched_biobert |      17 | ner_jsl_enriched_biobert         |      22 | ner_risk_factors_biobert   |
|       6 | ner_bionlp_biobert            |      12 | ner_diseases_biobert      |         |                                  |         |                            |


- **BertForTokenClassification Clinical NER models**

|    | model_name                         |
|---:|:-----------------------------------|
|  1 | bert_token_classifier_ner_ade      |
|  2 | bert_token_classifier_ner_clinical |
|  3 | bert_token_classifier_ner_deid     |
|  4 | bert_token_classifier_ner_drugs    |
|  5 | bert_token_classifier_ner_jsl      |
|  6 | bert_token_classifier_ner_jsl_slim |
|  7 | bert_token_classifier_ner_bionlp   |
|  8 | bert_token_classifier_ner_bacteria |
|  9 | bert_token_classifier_ner_anatomy  |
| 10 | bert_token_classifier_ner_cellular |
| 11 | bert_token_classifier_ner_chemprot |
| 12 | bert_token_classifier_ner_chemicals|
| 13 | bert_token_classifier_drug_development_trials   |



**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")\
  .setLabelCasing("upper")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]

In [0]:
#checking the stages in pipeline
model.stages

Out[3]: [DocumentAssembler_7e44b7fe76fc,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_0eadcaec4d11,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NerConverter_a7a10449786e]

In [0]:
#getting the classes in pretrained model
clinical_ner.getClasses()

Out[4]: ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [0]:
#extracting the embedded default param values
clinical_ner.extractParamMap()

Out[5]: {Param(parent='MedicalNerModel_1a8637089929', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_1a8637089929', name='labelCasing', doc='Setting all labels of the NER models upper/lower case. values upper|lower'): 'upper',
 Param(parent='MedicalNerModel_1a8637089929', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_1a8637089929', name='batchSize', doc='Size of every batch'): 64,
 Param(parent='MedicalNerModel_1a8637089929', name='classes', doc='get the tags used to trained this MedicalNerModel'): ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST'],
 Param(parent='MedicalNerModel_1a8637089929', name='inputCols', doc='previous annotations columns, if renamed'): ['sentence',
 'token',
 'embeddings'],
 Param(parent='MedicalNerModel_1a8637089929', name='outputCol', doc='output annotation column. can be left default.'): 'ner',
 Param(parent='MedicalNerModel_1a8637089929', name='storageRef', doc='unique reference name for identification'): 'clinical'}

In [0]:
#checking the embeddings
clinical_ner.getStorageRef()

Out[6]: 'clinical'

In [0]:
from sparknlp_jsl.compatibility import Compatibility 
import pandas as pd

compatibility = Compatibility(spark)

models = compatibility.findVersion('ner') 

models_df = pd.DataFrame([dict(x) for x in list(models)])
models_df

Out[7]:

,name,sparkVersion,version,language,date,readyToUse
0,nerdl_tumour_demo,2,1.7.3,en,2018-12-19T16:52:37.735,true
1,nercrf_tumour_demo,2,1.7.3,en,2018-12-19T17:23:53.776,true
2,nerdl_tumour_demo,2.4,1.8.0,en,2018-12-22T04:21:25.574,true
3,nercrf_tumour_demo,2.4,1.8.0,en,2018-12-22T04:46:26.992,true
4,nercrf_deid,2.4,1.8.0,en,2018-12-23T00:44:17.698,true
...,...,...,...,...,...,...
453,ner_anatomy_biobert_pipeline,3.0,3.4.1,en,2022-03-21T14:43:26.641,true
454,ner_ade_healthcare_pipeline,3.0,3.4.1,en,2022-03-22T10:16:20.015,true
455,ner_ade_clinical_pipeline,3.0,3.4.1,en,2022-03-21T14:55:30.624,true
456,ner_deid_subentity,3.0,3.4.2,pt,2022-04-13T09:04:03.338,true


In [0]:
#downloading the sample dataset
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

dbutils.fs.cp("file:/databricks/driver/pubmed_sample_text_small.csv", "dbfs:/") 

Out[8]: True

In [0]:
%fs ls file:/databricks/driver 

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/preload_class.lst,preload_class.lst,813069
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/display_result.html,display_result.html,2398


In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("dbfs:/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=80)

+--------------------------------------------------------------------------------+
 text|
+--------------------------------------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwar...|
BACKGROUND: At present, it is one of the most important issues for the treatm...|
OBJECTIVE: To investigate the relationship between preoperative atrialfibrill...|
Combined EEG/fMRI recording has been used to localize the generators of EEGev...|
Kohlschutter syndrome is a rare neurodegenerative disorder presenting withint...|
Statistical analysis of neuroimages is commonly approached with intergroupcom...|
The synthetic DOX-LNA conjugate was characterized by proton nuclear magneticr...|
Our objective was to compare three different methods of blood pressuremeasure...|
We conducted a phase II study to assess the efficacy and tolerability ofirino...|
"""Monomeric sarcosine oxidase (MSOX) is a flavoenzyme that catalyzes the oxi...|
We presented the tachinid fly Exorista japonica with moving host models: afre...|
The literature dealing with the water conducting properties of sapwood xylem ...|
A novel approach to synthesize chitosan-O-isopropyl-5'-O-d4T monophosphatecon...|
An HPLC-ESI-MS-MS method has been developed for the quantitative determinatio...|
The localizing and lateralizing values of eye and head ictal deviations durin...|
OBJECTIVE: To evaluate the effectiveness and acceptability of expectantmanage...|
For the construction of new combinatorial libraries, a lead compound was crea...|
We report the results of a screen for genetic association with urinary arseni...|
Intraparenchymal pericatheter cyst is rarely reported. Obstruction in thevent...|
It is known that patients with Klinefelter's syndrome are inclined to develop...|
+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
pubMedDF.printSchema()

root
-- text: string (nullable = true)

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 48, 106,...|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 67, 79, ...|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 130, 144...|
Combined EEG/fMRI...|[{document, 0, 16...|[{document, 0, 29...|[{token, 0, 7, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 26, C...|
Kohlschutter synd...|[{document, 0, 25...|[{document, 0, 20...|[{token, 0, 11, K...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 20, K...|
Statistical analy...|[{document, 0, 10...|[{document, 0, 22...|[{token, 0, 10, S...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 34, S...|
The synthetic DOX...|[{document, 0, 57...|[{document, 0, 10...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 30, T...|
Our objective was...|[{document, 0, 24...|[{document, 0, 19...|[{token, 0, 2, Ou...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 56, 80, ...|
We conducted a ph...|[{document, 0, 14...|[{document, 0, 24...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 28, ...|
"""Monomeric sarc...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 2, ""...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 36, M...|
We presented the ...|[{document, 0, 12...|[{document, 0, 34...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 13, 46, ...|
The literature de...|[{document, 0, 16...|[{document, 0, 13...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 154, 166...|
A novel approach ...|[{document, 0, 64...|[{document, 0, 97...|[{token, 0, 0, A,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 99, 124,...|
An HPLC-ESI-MS-MS...|[{document, 0, 90...|[{document, 0, 24...|[{token, 0, 1, An...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 23, A...|
The localizing an...|[{document, 0, 72...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 50, 70, ...|
OBJECTIVE: To eva...|[{document, 0, 13...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 97, 143,...|
For the construct...|[{document, 0, 32...|[{document, 0, 20...|[{token, 0, 2, Fo...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 24, 50, ...|
We report the res...|[{document, 0, 13...|[{document, 0, 31...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 25, 32, ...|
Intraparenchymal ...|[{document, 0, 10...|[{document, 0, 53...|[{token, 0, 15, I...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 33, I...|
It is known that ...|[{document, 0, 34...|[{document, 0, 14...|[{token, 0, 1, It...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 31, 52, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('token.result','ner.result').show(truncate=80)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 result| result|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
[The, human, KCNJ9, (, Kir, 3.3, ,, GIRK3, ), is, a, member, of, the, G-prote...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, I-TREATMENT...|
[BACKGROUND, :, At, present, ,, it, is, one, of, the, most, important, issues...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, B-PRO...|
[OBJECTIVE, :, To, investigate, the, relationship, between, preoperative, atr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O,...|
[Combined, EEG/fMRI, recording, has, been, used, to, localize, the, generator...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TEST,...|
[Kohlschutter, syndrome, is, a, rare, neurodegenerative, disorder, presenting...|[B-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PR...|
[Statistical, analysis, of, neuroimages, is, commonly, approached, with, inte...|[B-TEST, I-TEST, I-TEST, I-TEST, O, O, O, O, B-TREATMENT, O, O, O, O, O, O, O...|
[The, synthetic, DOX-LNA, conjugate, was, characterized, by, proton, nuclear,...|[B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, I...|
[Our, objective, was, to, compare, three, different, methods, of, blood, pres...|[O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, O, O, B-TEST, I-T...|
[We, conducted, a, phase, II, study, to, assess, the, efficacy, and, tolerabi...|[O, O, B-TEST, I-TEST, I-TEST, I-TEST, O, O, B-TREATMENT, I-TREATMENT, O, B-T...|
[""", Monomeric, sarcosine, oxidase, (, MSOX, ), is, a, flavoenzyme, that, ca...|[O, B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATM...|
[We, presented, the, tachinid, fly, Exorista, japonica, with, moving, host, m...|[O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, ...|
[The, literature, dealing, with, the, water, conducting, properties, of, sapw...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PR...|
[A, novel, approach, to, synthesize, chitosan-O-isopropyl-5'-O-d4T, monophosp...|[O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, B-PROBLEM, I-P...|
[An, HPLC-ESI-MS-MS, method, has, been, developed, for, the, quantitative, de...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, B-TREA...|
[The, localizing, and, lateralizing, values, of, eye, and, head, ictal, devia...|[O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, B-PROBLEM, I-PROBLE...|
[OBJECTIVE, :, To, evaluate, the, effectiveness, and, acceptability, of, expe...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PR...|
[For, the, construction, of, new, combinatorial, libraries, ,, a, lead, compo...|[O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROB...|
[We, report, the, results, of, a, screen, for, genetic, association, with, ur...|[O, O, O, O, O, B-TEST, I-TEST, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, ...|
[Intraparenchymal, pericatheter, cyst, is, rarely, reported, ., Obstruction, ...|[B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM...|
[It, is, known, that, patients, with, Klinefelter's, syndrome, are, inclined,...|[O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, I-PRO...|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
result_df= result.select(F.explode(F.arrays_zip(result.token.result, result.ner.result)).alias("cols"))\
                 .select(F.expr("cols['0']").alias("token"),
                         F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+----------------+---------+
 token|ner_label|
+----------------+---------+
 A| O|
 numerical| O|
 method| O|
 was| O|
 developed| O|
 using| O|
 geometrical| O|
 optics| O|
 to| O|
 predict| O|
far-fieldoptical|B-PROBLEM|
 scattering|I-PROBLEM|
 from| O|
 particles|B-PROBLEM|
 that| O|
 are| O|
 symmetric| O|
 about| O|
 the| O|
 optic| O|
 axis| O|
 .| O|
 Thediffractive| O|
 component| O|
 of| O|
 scattering| O|
 is| O|
 calculated| O|
 and| O|
 combined| O|
 with| O|
 thereflective| O|
 and| O|
 refractive| O|
 components| O|
 to| O|
 give| O|
 the| O|
 total| O|
 scattering| O|
 pattern| O|
 .| O|
 Thephase| O|
 terms| O|
 of| O|
 the| O|
 scattered| O|
 light| O|
 are| O|
 calculated| O|
+----------------+---------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |12587|
I-PROBLEM |1737 |
B-PROBLEM |1008 |
I-TREATMENT|789 |
B-TREATMENT|655 |
I-TEST |573 |
B-TEST |457 |
+-----------+-----+

In [0]:
result.select('ner_chunk').take(1)

Out[16]: [Row(ner_chunk=[Row(annotatorType='chunk', begin=69, end=95, result='far-fieldoptical scattering', metadata={'sentence': '0', 'chunk': '0', 'entity': 'PROBLEM', 'confidence': '0.71915'}, embeddings=[]), Row(annotatorType='chunk', begin=102, end=110, result='particles', metadata={'sentence': '0', 'chunk': '1', 'entity': 'PROBLEM', 'confidence': '0.9928'}, embeddings=[]), Row(annotatorType='chunk', begin=380, end=388, result='themethod', metadata={'sentence': '3', 'chunk': '2', 'entity': 'TREATMENT', 'confidence': '0.954'}, embeddings=[]), Row(annotatorType='chunk', begin=415, end=434, result='a spherical particle', metadata={'sentence': '3', 'chunk': '3', 'entity': 'PROBLEM', 'confidence': '0.6218333'}, embeddings=[]), Row(annotatorType='chunk', begin=462, end=484, result='toMie scattering theory', metadata={'sentence': '3', 'chunk': '4', 'entity': 'TREATMENT', 'confidence': '0.5957667'}, embeddings=[]), Row(annotatorType='chunk', begin=576, end=603, result='small-amplitude oscillations', metadata={'sentence': '5', 'chunk': '5', 'entity': 'PROBLEM', 'confidence': '0.6975'}, embeddings=[]), Row(annotatorType='chunk', begin=653, end=672, result='the numerical method', metadata={'sentence': '5', 'chunk': '6', 'entity': 'TREATMENT', 'confidence': '0.55516666'}, embeddings=[]), Row(annotatorType='chunk', begin=675, end=713, result='Numerical data fromspheroidal particles', metadata={'sentence': '6', 'chunk': '7', 'entity': 'PROBLEM', 'confidence': '0.729025'}, embeddings=[]), Row(annotatorType='chunk', begin=719, end=741, result='hemispherical particles', metadata={'sentence': '6', 'chunk': '8', 'entity': 'PROBLEM', 'confidence': '0.78104997'}, embeddings=[]), Row(annotatorType='chunk', begin=771, end=795, result='ofhemispherical particles', metadata={'sentence': '7', 'chunk': '9', 'entity': 'PROBLEM', 'confidence': '0.60539997'}, embeddings=[]), Row(annotatorType='chunk', begin=800, end=812, result='a calibration', metadata={'sentence': '7', 'chunk': '10', 'entity': 'TEST', 'confidence': '0.75925'}, embeddings=[]), Row(annotatorType='chunk', begin=827, end=875, result='intensity-type opticalparticle-sizing instruments', metadata={'sentence': '7', 'chunk': '11', 'entity': 'TREATMENT', 'confidence': '0.7364667'}, embeddings=[])])]

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------------------------------------+---------+
chunk |ner_label|
+-----------------------------------------------------------+---------+
the G-protein-activated inwardly rectifying potassium (GIRK|TREATMENT|
the genomicorganization |TREATMENT|
a candidate gene forType II diabetes mellitus |PROBLEM |
byapproximately |TREATMENT|
single nucleotide polymorphisms |TREATMENT|
aVal366Ala substitution |PROBLEM |
an 8 base-pair |PROBLEM |
insertion/deletion |PROBLEM |
Ourexpression studies |TEST |
the transcript in various humantissues |PROBLEM |
fat andskeletal muscle |PROBLEM |
furtherstudies |PROBLEM |
the KCNJ9 protein |TREATMENT|
evaluation |TEST |
Type II diabetes |PROBLEM |
the treatment |TREATMENT|
breast cancer |PROBLEM |
the standard therapy |TREATMENT|
anthracyclines |TREATMENT|
taxanes |TREATMENT|
+-----------------------------------------------------------+---------+
only showing top 20 rows

### with LightPipelines

In [0]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl ,  creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 . 
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 . 

Out[18]:

,chunks,begin,end,sentence_id,entities
0,gestational diabetes mellitus,40,68,0,PROBLEM
1,subsequent type two diabetes mellitus,118,154,0,PROBLEM
2,T2DM,158,161,0,PROBLEM
3,HTG-induced pancreatitis,187,210,0,PROBLEM
4,an acute hepatitis,268,285,0,PROBLEM
5,polyuria,326,333,0,PROBLEM
6,poor appetite,337,349,0,PROBLEM
7,vomiting,357,364,0,PROBLEM
8,metformin,380,388,1,TREATMENT
9,glipizide,392,400,1,TREATMENT


### NER Visualization

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

# Change color of an entity label

#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter

# visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])
  
displayHTML(ner_vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation , and associated with an acute hepatitis PROBLEM , presented with a one-week history of polyuria PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation . Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were : serum glucose TEST 111 mg/dl , creatinine TEST 0.4 mg/dL , triglycerides TEST 508 mg/dL , total cholesterol TEST 122 mg/dL , and venous pH TEST 7.27 .

## NER JSL Model

Let's show an example of `ner_jsl` model that has about 80 labels by changing just only the model name.

**Entities**

| | | | | |
|-|-|-|-|-|
|Injury_or_Poisoning|Direction|Test|Admission_Discharge|Death_Entity|
|Relationship_Status|Duration|Respiration|Hyperlipidemia|Birth_Entity|
|Age|Labour_Delivery|Family_History_Header|BMI|Temperature|
|Alcohol|Kidney_Disease|Oncological|Medical_History_Header|Cerebrovascular_Disease|
|Oxygen_Therapy|O2_Saturation|Psychological_Condition|Heart_Disease|Employment|
|Obesity|Disease_Syndrome_Disorder|Pregnancy|ImagingFindings|Procedure|
|Medical_Device|Race_Ethnicity|Section_Header|Symptom|Treatment|
|Substance|Route|Drug_Ingredient|Blood_Pressure|Diet|
|External_body_part_or_region|LDL|VS_Finding|Allergen|EKG_Findings|
|Imaging_Technique|Triglycerides|RelativeTime|Gender|Pulse|
|Social_History_Header|Substance_Quantity|Diabetes|Modifier|Internal_organ_or_component|
|Clinical_Dept|Form|Drug_BrandName|Strength|Fetus_NewBorn|
|RelativeDate|Height|Test_Result|Sexually_Active_or_Sexual_Orientation|Frequency|
|Time|Weight|Vaccine|Vital_Signs_Header|Communicable_Disease|
|Dosage|Overweight|Hypertension|HDL|Total_Cholesterol|
|Smoking|Date||||

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
  .setInputCols(["document"]) \
  .setOutputCol("sentence") 

#sentenceDetector = SentenceDetector()\
#   .setInputCols(["document"])\
#   .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("jsl_ner")\
  .setLabelCasing("upper")

jsl_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "jsl_ner"]) \
  .setOutputCol("jsl_ner_chunk")

jsl_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

jsl_ner_model = jsl_ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]

In [0]:
print (text)

jsl_light_model = LightPipeline(jsl_ner_model)

jsl_light_result = jsl_light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in jsl_light_result[0]['jsl_ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

jsl_df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

jsl_df.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 . 

Out[21]:

,chunks,begin,end,sentence_id,entities
0,28-year-old,3,13,0,AGE
1,female,15,20,0,GENDER
2,gestational diabetes mellitus,40,68,0,DIABETES
3,eight years prior,80,96,0,RELATIVEDATE
4,subsequent,118,127,0,MODIFIER
5,type two diabetes mellitus,129,154,0,DIABETES
6,T2DM,158,161,0,DIABETES
7,HTG-induced pancreatitis,187,210,0,DISEASE_SYNDROME_DISORDER
8,three years prior,212,228,0,RELATIVEDATE
9,acute,271,275,0,MODIFIER


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(jsl_light_result[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

A 28-year-old AGE female GENDER with a history of gestational diabetes mellitus DIABETES diagnosed eight years prior RELATIVEDATE to presentation and subsequent MODIFIER type two diabetes mellitus DIABETES ( T2DM DIABETES ), one prior episode of HTG-induced pancreatitis DISEASE_SYNDROME_DISORDER three years prior RELATIVEDATE to presentation , and associated with an acute MODIFIER hepatitis DISEASE_SYNDROME_DISORDER , presented with a one-week DURATION history of polyuria SYMPTOM , poor appetite SYMPTOM , and vomiting SYMPTOM . She GENDER was on metformin DRUG_INGREDIENT , glipizide DRUG_INGREDIENT , and dapagliflozin DRUG_INGREDIENT for T2DM DRUG_INGREDIENT and atorvastatin DRUG_INGREDIENT and gemfibrozil DRUG_INGREDIENT for HTG DRUG_INGREDIENT . She GENDER had been on dapagliflozin DRUG_INGREDIENT for six months DURATION at the time of presentation . Physical examination on presentation was significant for dry oral mucosa SYMPTOM ; significantly , her GENDER abdominal EXTERNAL_BODY_PART_OR_REGION examination was benign with no tenderness SYMPTOM , guarding SYMPTOM , or rigidity SYMPTOM . Pertinent laboratory findings on admission ADMISSION_DISCHARGE were : serum glucose TEST 111 mg/dl TEST_RESULT , creatinine TEST 0.4 mg/dL TEST_RESULT , triglycerides 508 mg/dL TRIGLYCERIDES , total cholesterol 122 mg/dL TOTAL_CHOLESTEROL , and venous pH TEST 7.27 TEST_RESULT .

## Posology NER

**Entities**

- DOSAGE
- DRUG
- DURATION
- FORM
- FREQUENCY
- ROUTE

In [0]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\
    .setLabelCasing("upper")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# greedy model
posology_ner_greedy = MedicalNerModel.pretrained("ner_posology_greedy", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_greedy")

ner_converter_greedy = NerConverter()\
    .setInputCols(["sentence","token","ner_greedy"])\
    .setOutputCol("ner_chunk_greedy")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    posology_ner_greedy,
    ner_converter_greedy])

empty_data = spark.createDataFrame([[""]]).toDF("text")
posology_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][OK!]
ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]

In [0]:
posology_ner.getClasses()

Out[24]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
posology_result = posology_model.transform(pubMedDF.limit(100))

In [0]:
posology_result.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 52, 122,...|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 167, 180...|[{named_entity, 0...|[{chunk, 167, 180...|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Combined EEG/fMRI...|[{document, 0, 16...|[{document, 0, 29...|[{token, 0, 7, Co...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Kohlschutter synd...|[{document, 0, 25...|[{document, 0, 20...|[{token, 0, 11, K...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
Statistical analy...|[{document, 0, 10...|[{document, 0, 22...|[{token, 0, 10, S...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
The synthetic DOX...|[{document, 0, 57...|[{document, 0, 10...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 576, 578...|[{named_entity, 0...|[{chunk, 432, 446...|
Our objective was...|[{document, 0, 24...|[{document, 0, 19...|[{token, 0, 2, Ou...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 174, 185...|[{named_entity, 0...|[{chunk, 174, 185...|
We conducted a ph...|[{document, 0, 14...|[{document, 0, 24...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 70, 81, ...|[{named_entity, 0...|[{chunk, 70, 81, ...|
"""Monomeric sarc...|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 2, ""...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 138, 144...|[{named_entity, 0...|[{chunk, 0, 36, "...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [0]:
posology_result.printSchema()

root
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_greedy: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk_greedy: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
posology_result = posology_result.withColumn("id", monotonically_increasing_id())

posology_result.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy| id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
The human KCNJ9 (...|[{document, 0, 95...|[{document, 0, 12...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 52, 122,...| 0|
BACKGROUND: At pr...|[{document, 0, 14...|[{document, 0, 19...|[{token, 0, 9, BA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 167, 180...|[{named_entity, 0...|[{chunk, 167, 180...| 1|
OBJECTIVE: To inv...|[{document, 0, 15...|[{document, 0, 14...|[{token, 0, 8, OB...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []| 2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 3 rows

In [0]:
posology_result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[A, numerical, method, was, developed, using, geometrical, optics, to, predict, far-fieldoptical,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Recording, of, the, esophageal, ECG, is, important, in, differentiating, between, differenttypes...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Colicin, E5, cleaves, tRNAs, for, Tyr, ,, His, ,, Asn, and, Asp, in, their, anticodons, to, abol...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Recombinant, adenoviruses, (, rAd, ),, widely, used, as, vectors, for, gene, therapy, ,, aregene...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[To, understand, its, potential, to, cause, invasive, disease, ,, the, genome, of, Mycoplasmacani...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[OBJECTIVE, :, To, review, a, group, of, Australian, women, presenting, to, a, reproductiveendocr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[AIM, :, Understanding, factors, that, contribute, to, changes, in, arterial, stiffness, over, ti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[In, the, present, study, ,, the, pharmacological, effects, of, BmK, AS, ,, a, beta-like, scorpio...|[O, O, O, O, O, O, O, O, O, B-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[AIM, :, To, describe, the, risk, factors, and, molecular, epidemiology, of, nosocomialbloodstrea...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Although, the, risks, smoking, poses, to, health, are, now, well, known, ,, many, young, people,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Successful, long-term, myocardial, preservation, is, dependent, on, optimizingconditions, during...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Pulmonary, alveolar, macrophages, lavaged, from, tobacco, smokers, release, increasedlevels, of,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Prenatal, care, is, focused, on, improving, the, health, of, women, and, babies, ., Traditionalm...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Controversy, exists, regarding, the, safety, of, intravenous, dipyridamole, in, patientswith, se...|[O, O, O, O, O, O, B-ROUTE, B-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Children, with, a, family, history, of, thrombophilia, and/or, thrombosis, are, oftenreferred, t...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, mucosal, immune, system, is, constantly, exposed, to, antigen, ,, whether, it, be, foodanti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Urinary, concretions, ,, particularly, in, the, upper, urinary, tract, ,, occur, in, otherwise, ...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[In, this, study, ,, the, biosorption, of, nickel(II, ), ion, on, deactivated, protonated, yeast,...|[O, O, O, O, O, O, O, O

In [0]:
from pyspark.sql import functions as F

posology_result_df = posology_result.select(F.explode(F.arrays_zip(posology_result.token.result, posology_result.ner.result )).alias("cols")) \
                                    .select(F.expr("cols['0']").alias("token"),
                                            F.expr("cols['1']").alias("ner_label"))\
                                    .filter("ner_label!='O'")

posology_result_df.show(20, truncate=100)

+---------------+-----------+
 token| ner_label|
+---------------+-----------+
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 vinorelbine| B-DRUG|
 vinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 Vinorelbinewas| B-DRUG|
 25| B-STRENGTH|
 mg/m(2| I-STRENGTH|
 intravenously| B-ROUTE|
 on|B-FREQUENCY|
 days|I-FREQUENCY|
thatvinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 DOX| B-DRUG|
 trandolapril| B-DRUG|
 losartan| B-DRUG|
 3-week| B-DURATION|
 placebo| B-DRUG|
+---------------+-----------+
only showing top 20 rows

In [0]:
posology_greedy_result_df = posology_result.select(F.explode(F.arrays_zip(posology_result.token.result,
                                                                          posology_result.ner_greedy.result)).alias("cols")) \
                                           .select(F.expr("cols['0']").alias("token"),
                                                    F.expr("cols['1']").alias("ner_label"))\
                                           .filter("ner_label!='O'")

posology_greedy_result_df.show(20, truncate=100)

+---------------------------+---------+
 token|ner_label|
+---------------------------+---------+
 lackedprotein| B-DRUG|
 VI| I-DRUG|
 immobilized| B-DRUG|
 zinc| I-DRUG|
 affinity| I-DRUG|
 chromatography| I-DRUG|
 valuabletechnique| B-DRUG|
reproductiveendocrinologist| B-DRUG|
 dehydroepiandrosterone| B-DRUG|
 sulphate| I-DRUG|
 low-dose| B-DRUG|
 dexamethasone| I-DRUG|
 withethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
 acetate| I-DRUG|
 ethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
 acetate| I-DRUG|
 withethinyloestradiol| B-DRUG|
 cyproterone| B-DRUG|
+---------------------------+---------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk.result, 
                                                   posology_result.ner_chunk.begin, 
                                                   posology_result.ner_chunk.end, 
                                                   posology_result.ner_chunk.metadata)).alias("cols")) \
              .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"))\
                            .filter("ner_label!='O'")\
              .show(truncate=False)

+---+-----------+----------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+----------------+-----+----+---------+
1 |0 |anthracyclines |167 |180 |DRUG |
1 |0 |taxanes |186 |192 |DRUG |
1 |1 |vinorelbine |246 |256 |DRUG |
1 |1 |vinorelbine |386 |396 |DRUG |
1 |1 |anthracyclines |433 |446 |DRUG |
1 |1 |taxanes |452 |458 |DRUG |
1 |2 |Vinorelbinewas |470 |483 |DRUG |
1 |2 |25 mg/m(2 |517 |525 |STRENGTH |
1 |2 |intravenously |528 |540 |ROUTE |
1 |2 |on days |542 |548 |FREQUENCY|
1 |10 |thatvinorelbine |1263 |1277|DRUG |
1 |10 |anthracyclines |1380 |1393|DRUG |
1 |10 |taxanes |1399 |1405|DRUG |
6 |3 |DOX |576 |578 |DRUG |
7 |0 |trandolapril |174 |185 |DRUG |
7 |0 |losartan |191 |198 |DRUG |
7 |2 |3-week |313 |318 |DURATION |
7 |2 |placebo |320 |326 |DRUG |
7 |2 |2 mgtrandolapril|357 |372 |STRENGTH |
7 |2 |50 mg |377 |381 |STRENGTH |
+---+-----------+----------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk_greedy.result, 
                                                   posology_result.ner_chunk_greedy.begin, 
                                                   posology_result.ner_chunk_greedy.end, 
                                                   posology_result.ner_chunk_greedy.metadata)).alias("cols")) \
              .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"))\
                            .filter("ner_label!='O'")\
                            .show(truncate=False)

+---+-----------+----------------------------------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+----------------------------------------+-----+----+---------+
3 |4 |lackedprotein VI |539 |554 |DRUG |
3 |6 |immobilized zinc affinity chromatography|717 |756 |DRUG |
3 |10 |valuabletechnique |1229 |1245|DRUG |
5 |0 |reproductiveendocrinologist |65 |91 |DRUG |
5 |8 |dehydroepiandrosterone sulphate |972 |1002|DRUG |
5 |8 |low-dose dexamethasone |1021 |1042|DRUG |
5 |9 |withethinyloestradiol |1077 |1097|DRUG |
5 |9 |cyproterone acetate |1103 |1121|DRUG |
5 |13 |ethinyloestradiol |1556 |1572|DRUG |
5 |13 |cyproterone acetate |1578 |1596|DRUG |
5 |17 |withethinyloestradiol |1914 |1934|DRUG |
5 |17 |cyproterone acetate |1940 |1958|DRUG |
7 |4 |BmK |634 |636 |DRUG |
8 |0 |enterobacteria-positive blood cultures |226 |263 |DRUG |
8 |2 |bla(CTX-M |465 |473 |DRUG |
8 |6 |ofbroad-spectrum cephalosporins |975 |1005|DRUG |
8 |7 |imipenem |1046 |1053|DRUG |
8 |10 |broad-spectrumcephalosporins |1348 |1375|DRUG |
11 |5 |monoxide |930 |937 |DRUG |
13 |0 |intravenous dipyridamole |43 |66 |DRUG |
+---+-----------+----------------------------------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('ner_chunk').take(2)

Out[34]: [Row(ner_chunk=[]), Row(ner_chunk=[])]

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].result

Out[35]: 'anthracyclines'

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].metadata

Out[37]: {'sentence': '0', 'chunk': '0', 'entity': 'DRUG', 'confidence': '0.9988'}

### with LightPipelines

In [0]:
light_model = LightPipeline(posology_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

posology_light_result = light_model.annotate(text)

list(zip(posology_light_result['token'], posology_light_result['ner']))

Out[38]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
list(zip(posology_light_result['token'], posology_light_result['ner_greedy']))

Out[39]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DRUG'),
 ('capsule', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('Advil', 'I-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'I-DRUG'),
 ('mg', 'I-DRUG'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
posology_light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in posology_light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 
                   'entities':entities,
                   'begin': begin, 
                   'end': end})

df

Out[40]:

,chunks,entities,begin,end
0,1,DOSAGE,27,27
1,capsule,FORM,29,35
2,Advil,DRUG,40,44
3,for 5 days,DURATION,46,55
4,40 units,DOSAGE,126,133
5,insulin glargine,DRUG,138,153
6,at night,FREQUENCY,155,162
7,12 units,DOSAGE,166,173
8,insulin lispro,DRUG,178,191
9,with meals,FREQUENCY,193,202


In [0]:
chunks = []
entities = []
begin =[]
end = []

for n in posology_light_result[0]['ner_chunk_greedy']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

posology_result_greedy_df = pd.DataFrame({'chunks':chunks, 
                                          'entities':entities,
                                          'begin': begin, 
                                          'end': end})

posology_result_greedy_df.head(15)

Out[41]:

,chunks,entities,begin,end
0,1 capsule of Advil,DRUG,27,44
1,for 5 days,DURATION,46,55
2,40 units of insulin glargine,DRUG,126,153
3,at night,FREQUENCY,155,162
4,12 units of insulin lispro,DRUG,166,191
5,with meals,FREQUENCY,193,202
6,metformin 1000 mg,DRUG,210,226
7,two times a day,FREQUENCY,228,242
8,SGLT2 inhibitors,DRUG,273,288
9,for 3 months,DURATION,326,337


### NER Visualization

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(posology_light_result[0], label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

The patient was prescribed 1 DOSAGE capsule FORM of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

In [0]:
# ner_greedy

visualiser_greedy = NerVisualizer()

ner_greedy_vis = visualiser_greedy.display(posology_light_result[0], label_col='ner_chunk_greedy', document_col='document', return_html=True)

displayHTML(ner_greedy_vis)

The patient was prescribed 1 capsule of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine DRUG at night FREQUENCY , 12 units of insulin lispro DRUG with meals FREQUENCY , and metformin 1000 mg DRUG two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

## Writing a generic NER function

In [0]:
# Generic NER Function with LightPipeline

def get_light_model (embeddings, model_name = 'ner_clinical'):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      loaded_ner_model,
      ner_converter])

  model = nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

  return LightPipeline(model)

In [0]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  base_pipeline = Pipeline(stages=[
                    documentAssembler,
                    sentenceDetector,
                    tokenizer,
                    word_embeddings
                  ])

  return base_pipeline



def get_clinical_entities (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  base_pipeline = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_pipeline,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select('id',F.explode(F.arrays_zip(result.ner_chunk.result, 
                                                        result.ner_chunk.begin, 
                                                        result.ner_chunk.end, 
                                                        result.ner_chunk.metadata)).alias("cols")) \
                    .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['3']['entity']").alias("ner_label") )\
                    .filter("ner_label!='O'")

  return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+-----+---+--------------------+---------+
 id|sentence_id|begin|end| chunk|ner_label|
+---+-----------+-----+---+--------------------+---------+
 0| 0| 69| 95|far-fieldoptical ...| PROBLEM|
 0| 0| 102|110| particles| PROBLEM|
 0| 3| 380|388| themethod|TREATMENT|
 0| 3| 415|434|a spherical particle| PROBLEM|
 0| 3| 462|484|toMie scattering ...|TREATMENT|
 0| 5| 576|603|small-amplitude o...| PROBLEM|
 0| 5| 653|672|the numerical method|TREATMENT|
 0| 6| 675|713|Numerical data fr...| PROBLEM|
 0| 6| 719|741|hemispherical par...| PROBLEM|
 0| 7| 771|795|ofhemispherical p...| PROBLEM|
 0| 7| 800|812| a calibration| TEST|
 0| 7| 827|875|intensity-type op...|TREATMENT|
 1| 0| 13| 30| the esophageal ECG| TEST|
 1| 0| 90|100| tachycardia| PROBLEM|
 1| 1| 103|110| A method|TREATMENT|
 1| 1| 134|163|the Arzco pill el...|TREATMENT|
 1| 2| 174|186| The electrode|TREATMENT|
 1| 2| 236|254| atelemetric monitor| TEST|
 1| 3| 269|286| the esophageal ECG| TEST|
 1| 3| 343|379|transient, self-l...| PROBLEM|
+---+-----------+-----+---+--------------------+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+-----+----+--------------------+---------+
 id|sentence_id|begin| end| chunk|ner_label|
+---+-----------+-----+----+--------------------+---------+
 3| 3| 539| 554| lackedprotein VI| DRUG|
 5| 8| 972|1002|dehydroepiandrost...| DRUG|
 5| 8| 1021|1028| low-dose| DOSAGE|
 5| 8| 1030|1042| dexamethasone| DRUG|
 5| 9| 1077|1097|withethinyloestra...| DRUG|
 5| 9| 1103|1121| cyproterone acetate| DRUG|
 5| 15| 1556|1572| ethinyloestradiol| DRUG|
 5| 15| 1578|1596| cyproterone acetate| DRUG|
 5| 18| 1914|1934|withethinyloestra...| DRUG|
 5| 18| 1940|1958| cyproterone acetate| DRUG|
 7| 0| 53| 55| BmK| DRUG|
 7| 3| 634| 636| BmK| DRUG|
 8| 5| 975|1005|ofbroad-spectrum ...| DRUG|
 10| 1| 258| 271| blood-perfused| DRUG|
 10| 1| 297| 311| isolatedcircuit| DRUG|
 10| 3| 475| 546|aspartate/glutama...| DRUG|
 10| 3| 603| 652|andaspartate/glut...| DRUG|
 11| 5| 678| 687| Superoxide| DRUG|
 11| 5| 930| 937| monoxide| DRUG|
 12| 2| 237| 244| modelsin| DRUG|
+---+-----------+-----+----+--------------------+---------+
only showing top 20 rows

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0]['ner_chunk']:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
import pandas as pd

def get_light_result (light_model, text, chunk_name="ner_chunk"):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []
  sentence= []
  begin = []
  end = []

  for n in light_result[0][chunk_name]:
                  
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence']) 
      
    pd_df = pd.DataFrame({'sentence_id':sentence, 
                          'begin': begin, 
                          'end':end, 
                          'chunks':chunks,  
                          'entities':entities})
  return pd_df

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(posology_model)

get_clinical_entities_light (light_model, text)

Out[50]:

,chunks,entities
0,1,DOSAGE
1,capsule,FORM
2,Parol,DRUG
3,with meals,FREQUENCY
4,40 units,DOSAGE
5,insulin glargine,DRUG
6,at night,FREQUENCY
7,12 units,DOSAGE
8,insulin lispro,DRUG
9,with meals,FREQUENCY


In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'


embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

light_model = get_light_model (embeddings, model_name)

get_light_result (light_model, text, chunk_name="ner_chunk")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
Out[51]:

,sentence_id,begin,end,chunks,entities
0,0,27,27,1,DOSAGE
1,0,29,35,capsule,FORM
2,0,40,44,Parol,DRUG
3,0,46,55,with meals,FREQUENCY
4,1,126,133,40 units,DOSAGE
5,1,138,153,insulin glargine,DRUG
6,1,155,162,at night,FREQUENCY
7,1,166,173,12 units,DOSAGE
8,1,178,191,insulin lispro,DRUG
9,1,193,202,with meals,FREQUENCY


## PHI NER

**Entities**
- AGE
- CONTACT
- DATE
- ID
- LOCATION
- NAME
- PROFESSION

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_deid_large'

# deidentify_dl
# ner_deid_large

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

pd_ner_df = ner_df.toPandas()

ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
pd_ner_df.sample(20)

Out[53]:

,id,sentence_id,begin,end,chunk,ner_label
11,32,5,607,626,elevatedtemperatures,LOCATION
61,83,2,725,733,June 1995,DATE
12,33,0,145,149,Benin,LOCATION
28,45,10,1525,1528,9/29,DATE
22,45,1,417,420,2009,DATE
58,77,11,1392,1405,surgicallevel.,NAME
34,49,2,352,355,2005,DATE
60,83,2,709,720,January 1995,DATE
3,16,1,256,266,Scandinavia,LOCATION
4,21,1,294,304,East Africa,LOCATION


In [0]:
pd_ner_df.ner_label.value_counts()

Out[54]: LOCATION 24
DATE 22
NAME 14
AGE 7
PROFESSION 1
Name: ner_label, dtype: int64

## BioNLP (Cancer Genetics) NER

**Entities**

| | | |
|-|-|-|
|tissue_structure|Amino_acid|Simple_chemical|
|Organism_substance|Developing_anatomical_structure|Cell|
|Cancer|Cellular_component|Gene_or_gene_product|
|Immaterial_anatomical_entity|Organ|Organism|
|Pathological_formation|Organism_subdivision|Anatomical_system|
|Tissue|||

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+-----+---+----------------------+--------------------+
id |sentence_id|begin|end|chunk |ner_label |
+---+-----------+-----+---+----------------------+--------------------+
0 |0 |4 |8 |human |Organism |
0 |0 |17 |23 |Kir 3.3 |Gene_or_gene_product|
0 |0 |26 |30 |GIRK3 |Gene_or_gene_product|
0 |0 |92 |100|potassium |Simple_chemical |
0 |0 |103 |106|GIRK |Gene_or_gene_product|
0 |1 |188 |205|chromosome 1q21-23 |Cellular_component |
0 |5 |697 |704|pancreas |Organ |
0 |5 |740 |746|tissues |Tissue |
0 |5 |749 |770|fat andskeletal muscle|Tissue |
0 |6 |801 |805|KCNJ9 |Gene_or_gene_product|
0 |6 |940 |946|Type II |Gene_or_gene_product|
1 |0 |84 |96 |breast cancer |Cancer |
1 |0 |134 |141|patients |Organism |
1 |0 |167 |180|anthracyclines |Simple_chemical |
1 |0 |186 |192|taxanes |Simple_chemical |
1 |1 |246 |256|vinorelbine |Simple_chemical |
1 |1 |273 |280|patients |Organism |
1 |1 |309 |314|breast |Cancer |
1 |1 |386 |407|vinorelbine inpatients|Simple_chemical |
1 |1 |433 |446|anthracyclines |Simple_chemical |
+---+-----------+-----+---+----------------------+--------------------+
only showing top 20 rows

## NER Chunker
We can extract phrases that fits into a known pattern using the NER tags. NerChunker would be quite handy to extract entity groups with neighboring tokens when there is no pretrained NER model to address certain issues. Lets say we want to extract clinical findings and body parts together as a single chunk even if there are some unwanted tokens between.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_chunker = NerChunker()\
    .setInputCols(["sentence","ner"])\
    .setOutputCol("ner_chunk")\
    .setRegexParsers(["<DRUG>.*<FREQUENCY>"])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_chunker])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_chunker_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
posology_ner.getClasses()

Out[57]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
light_model = LightPipeline(ner_chunker_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[58]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('fro', 'O'),
 ('3', 'O'),
 ('months', 'O'),
 ('.', 'O')]

In [0]:
light_result["ner_chunk"]

Out[59]: ['insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day']

## Chunk Filterer
ChunkFilterer will allow you to filter out named entities by some conditions or predefined look-up lists, so that you can feed these entities to other annotators like Assertion Status or Entity Resolvers. It can be used with two criteria: isin and regex.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")
      
chunk_filterer = ChunkFilterer()\
      .setInputCols("sentence","ner_chunk")\
      .setOutputCol("chunk_filtered")\
      .setCriteria("isin")\
      .setWhiteList(['Advil','metformin', 'insulin lispro'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]

In [0]:
light_model = LightPipeline(chunk_filter_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

light_result.keys()

Out[61]: dict_keys(['document', 'ner_chunk', 'chunk_filtered', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
light_result['ner_chunk'] 

Out[62]: ['1',
 'capsule',
 'Advil',
 'for 5 days',
 '40 units',
 'insulin glargine',
 'at night',
 '12 units',
 'insulin lispro',
 'with meals',
 'metformin',
 '1000 mg',
 'two times a day',
 'SGLT2 inhibitors']

In [0]:
light_result["chunk_filtered"]

Out[63]: ['Advil', 'insulin lispro', 'metformin']

In [0]:
ner_model = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentence","token","embeddings")\
    .setOutputCol("ner")
      
chunk_filterer = ChunkFilterer()\
    .setInputCols("sentence","ner_chunk")\
    .setOutputCol("chunk_filtered")\
    .setCriteria("isin")\
    .setWhiteList(['severe fever','sore throat'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]

In [0]:
text = 'Patient with severe fever, severe cough, sore throat, stomach pain, and a headache.'

filter_df = spark.createDataFrame([[text]]).toDF('text')

chunk_filter_result = chunk_filter_model.transform(filter_df)

In [0]:
chunk_filter_result.select('ner_chunk.result', 'chunk_filtered.result').show(truncate=False)

+-------------------------------------------------------------------+---------------------------+
result |result |
+-------------------------------------------------------------------+---------------------------+
[severe fever, severe cough, sore throat, stomach pain, a headache]|[severe fever, sore throat]|
+-------------------------------------------------------------------+---------------------------+

## Changing entity labels with `NerConverterInternal()`

In [0]:
replace_dict = """Drug_BrandName,Drug
Frequency,Drug_Frequency
Dosage,Drug_Dosage
Strength,Drug_Strength
"""
with open('replace_dict.csv', 'w') as f:
    f.write(replace_dict)

In [0]:
%fs ls file:/databricks/driver

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/preload_class.lst,preload_class.lst,813069
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/replace_dict.csv,replace_dict.csv,87
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/display_result.html,display_result.html,2398


In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") 

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "jsl_ner"]) \
    .setOutputCol("jsl_ner_chunk")

jsl_ner_converter_internal = NerConverterInternal()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("replaced_ner_chunk")\
    .setReplaceDictResource("file:/databricks/driver/replace_dict.csv","text", {"delimiter":","})
      
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter,
    jsl_ner_converter_internal
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_converter_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[ | ][OK!]

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text, chunk_name="ner_chunk"):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0][chunk_name]:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(ner_converter_model)

jsl_ner_chunk_df = get_clinical_entities_light (light_model, text, chunk_name='jsl_ner_chunk')
replaced_ner_chunk_df = get_clinical_entities_light (light_model, text, chunk_name='replaced_ner_chunk')
pd.concat([jsl_ner_chunk_df, replaced_ner_chunk_df.iloc[:,1:].rename(columns= {'entities':'replaced'})], axis=1)


Out[70]:

,chunks,entities,replaced
0,1 capsule,DOSAGE,DOSAGE
1,Parol,DRUG_BRANDNAME,DRUG_BRANDNAME
2,He,GENDER,GENDER
3,endocrinology service,CLINICAL_DEPT,CLINICAL_DEPT
4,she,GENDER,GENDER
5,discharged,ADMISSION_DISCHARGE,ADMISSION_DISCHARGE
6,40 units,DOSAGE,DOSAGE
7,insulin glargine,DRUG_INGREDIENT,DRUG_INGREDIENT
8,at night,FREQUENCY,FREQUENCY
9,12 units,DOSAGE,DOSAGE


In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(ner_converter_model)

jsl_ner_chunk_df = get_light_result (light_model, text, chunk_name='jsl_ner_chunk')
replaced_ner_chunk_df = get_light_result (light_model, text, chunk_name='replaced_ner_chunk')
pd.concat([jsl_ner_chunk_df, replaced_ner_chunk_df.iloc[:,-1:].rename(columns= {'entities':'replaced'})], axis=1)

Out[71]:

,sentence_id,begin,end,chunks,entities,replaced
0,0,27,35,1 capsule,DOSAGE,DOSAGE
1,0,40,44,Parol,DRUG_BRANDNAME,DRUG_BRANDNAME
2,1,59,60,He,GENDER,GENDER
3,1,78,98,endocrinology service,CLINICAL_DEPT,CLINICAL_DEPT
4,1,104,106,she,GENDER,GENDER
5,1,112,121,discharged,ADMISSION_DISCHARGE,ADMISSION_DISCHARGE
6,1,126,133,40 units,DOSAGE,DOSAGE
7,1,138,153,insulin glargine,DRUG_INGREDIENT,DRUG_INGREDIENT
8,1,155,162,at night,FREQUENCY,FREQUENCY
9,1,166,173,12 units,DOSAGE,DOSAGE


## Downloading Pretrained Models

- When we use `.pretrained` method, model is downloaded to  a folder named `cache_pretrained` automatically and it is loaded from thit folder if you run it again.

- In order to download the models manually to any folder, you can follow the steps below. In this case you should use `.load()` method.

  - Install AWS CLI to your local computer following the steps [here](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html) for Linux and [here](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-mac.html) for MacOS.

  - Then configure your AWS credentials.

  - Go to models hub and look for the model you need.

  - Select the model you found and you will see the model card that shows all the details about that model.

  - Hover the Download button on that page and you will see the download link from the S3 bucket. 

  - Just use AWS CLI like follows:

```
!aws s3 cp --region us-east-2 s3://auxdata.johnsnowlabs.com/clinical/models/ner_jsl_en_3.1.0_2.4_1624566960534.zip .
```

# Training a Clinical NER (NCBI Disease Dataset)

**WARNING:** You should use TensorFlow version 2.3 for training a Clinical Ner.

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[72]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[74]: 3266

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
conll_data.select("label.result").distinct().count()

Out[76]: 1537

In [0]:
'''
As you can see, there are too many `O` labels in the dataset. 
To make it more balanced, we can drop the sentences have only O labels.
(`c>1` means we drop all the sentences that have no valuable labels other than `O`)
'''

'''
conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)
'''

Out[77]: '\nconll_data = conll_data.withColumn(\'unique\', F.array_distinct("label.result")) .withColumn(\'c\', F.size(\'unique\')) .filter(F.col(\'c\')>1)\n\nconll_data.select(F.explode(F.arrays_zip(\'token.result\',\'label.result\')).alias("cols")) .select(F.expr("cols[\'0\']").alias("token"),\n F.expr("cols[\'1\']").alias("ground_truth")) .groupBy(\'ground_truth\') .count() .orderBy(\'count\', ascending=False) .show(100,truncate=False)\n'

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
test_data = CoNLL().readDataset(spark, "file:/databricks/driver/NER_NCBIconlltest.txt")

test_data = clinical_embeddings.transform(test_data)

test_data.write.parquet('dbfs/NER_NCBIconlltest.parquet')

In [0]:
%fs ls file:/databricks/driver

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/preload_class.lst,preload_class.lst,813069
file:/databricks/driver/file:/,file:/,4096
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/NER_NCBIconlltrain.txt,NER_NCBIconlltrain.txt,1091488
file:/databricks/driver/replace_dict.csv,replace_dict.csv,87
file:/databricks/driver/NER_NCBIconlltest.txt,NER_NCBIconlltest.txt,244358
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/pubmed_sample_text_small.csv,pubmed_sample_text_small.csv,9363435
file:/databricks/driver/logs/,logs/,4096


## MedicalNER Graph

**WARNING:** For training an NER model with custom graph, please use TensorFlow version 2.3

In [0]:
import tensorflow
from sparknlp_jsl.training import tf_graph

In [0]:
tensorflow.__version__

Out[92]: '2.3.0'

Firstly, we will create graph and log folder.

In [0]:
%fs mkdirs file:/dbfs/ner/ner_logs

res9: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/ner/medical_ner_graphs

res10: Boolean = true

In [0]:
%fs ls file:/dbfs/ner

path,name,size
file:/dbfs/ner/medical_ner_graphs/,medical_ner_graphs/,4096
file:/dbfs/ner/ner_logs/,ner_logs/,4096


We created ner log and graph files. 
Now, we will create graph and fit the model.

In [0]:
tf_graph.print_model_params("ner_dl")
tf_graph.build("ner_dl", 
               build_params={"embeddings_dim": 200, 
                             "nchars": 85, 
                             "ntags": 12, 
                             "is_medical": 1},
               model_location="/dbfs/ner/medical_ner_graphs", 
               model_filename="auto")

ner_dl parameters.
Parameter Required Default value Description
ntags yes - Number of tags.
embeddings_dim no 200 Embeddings dimension.
nchars no 100 Number of chars.
lstm_size no 128 Number of LSTM units.
gpu_device no 0 Device for training.
is_medical no 0 Build a Medical Ner graph.
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_12_200_128_85.pb

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

Out[94]: '\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py\n\n!pip -q install tensorflow==1.15.0\n\nimport create_graph\n\nntags = 3 # number of labels\nembeddings_dim = 200\nnchars =83\n\ncreate_graph.create_graph(ntags, embeddings_dim, nchars)\n'

In [0]:
%fs ls file:/dbfs/ner/medical_ner_graphs/

path,name,size
file:/dbfs/ner/medical_ner_graphs/blstm_12_200_128_85.pb,blstm_12_200_128_85.pb,1767991


In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(20)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setTestDataset("/dbfs/NER_NCBIconlltest.parquet")\
  .setOutputLogsPath('dbfs:/ner/ner_logs')\
  .setGraphFolder('dbfs:/ner/medical_ner_graphs')\
  .setUseBestModel(True)\
  .setEarlyStoppingCriterion(0.04)\
  .setEarlyStoppingPatience(3)\
  # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          nerTagger
 ])

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

In [0]:
#%sh rm -r /dbfs/ner/ner_logs/*

# remove existing logs

In [0]:
# 20 epochs 3 min
ner_model = ner_pipeline.fit(conll_data)

# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook in public folder to create a graph
# licensed users can also use 17.Graph_builder_for_DL_models.ipynb to create tf graphs easily.

`getTrainingClassDistribution()` parameter returns the distribution of labels used when training the NER model.

In [0]:
ner_model.stages[1].getTrainingClassDistribution()

Out[115]: {'O': 59262, 'B-Disease': 2465, 'I-Disease': 2903}

In [0]:
%sh cd /dbfs/ner/ && ls -la

total 16
drwxrwxrwx 2 root root 4096 Feb 19 17:12 .
drwxrwxrwx 2 root root 4096 Apr 26 10:59 ..
drwxrwxrwx 2 root root 4096 Apr 26 10:59 medical_ner_graphs
drwxrwxrwx 2 root root 4096 Apr 26 10:59 ner_logs

In [0]:
%sh cd /dbfs/ner/ner_logs && ls -lt

total 10
-rwxrwxrwx 1 root root 9541 Apr 26 11:48 MedicalNerApproach_ed50b82197cc.log

In [0]:
import os 
log_file= os.listdir("/dbfs/ner/ner_logs")[0]

with open (f"/dbfs/ner/ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: /databricks/driver/file:/local_disk0/tmp/sparknlp_tmp_1572610800230842971/blstm_12_200_128_85.pb
Training started - total epochs: 20 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 82 - training examples: 2589


Epoch 1/20 started, lr: 0.001, dataset size: 2589


Epoch 1/20 - 12.73s - loss: 468.2468 - avg training loss: 11.148733 - batches: 42
Quality on validation dataset (20.0%), validation examples = 517
time to finish evaluation: 2.06s
Total validation loss: 47.0076	Avg validation loss: 3.6160
label tp fp fn prec rec f1
I-Disease 288 85 356 0.772118 0.44720498 0.56637174
B-Disease 304 136 324 0.6909091 0.48407644 0.5692884
tp: 592 fp: 221 fn: 680 labels: 2
Macro-average prec: 0.7315135, rec: 0.46564072, f1: 0.5690536
Micro-average prec: 0.7281673, rec: 0.4654088, f1: 0.5678657
Quality on test dataset: 
time to finish evaluation: 1.84s
Total test loss: 61.2449	Avg test loss: 4.3746
label tp fp fn prec rec f1
I-Disease 282 97 507 0.7440633 0.35741445 0.48287675
B-Disease 303 167 405 0.64468086 0.4279661 0.51443124
tp: 585 fp: 264 fn: 912 labels: 2
Macro-average prec: 0.69437206, rec: 0.39269027, f1: 0.50166976
Micro-average prec: 0.68904597, rec: 0.39078155, f1: 0.4987212


Epoch 2/20 started, lr: 9.950249E-4, dataset size: 2589


Epoch 2/20 - 11.46s - loss: 169.91663 - avg training loss: 4.045634 - batches: 42
Quality on validation dataset (20.0%), validation examples = 517
time to finish evaluation: 1.57s
Total validation loss: 36.3788	Avg validation loss: 2.7984
label tp fp fn prec rec f1
I-Disease 442 235 202 0.6528804 0.6863354 0.66919
B-Disease 435 145 193 0.75 0.6926752 0.72019875
tp: 877 fp: 380 fn: 395 labels: 2
Macro-average prec: 0.7014402, rec: 0.6895053, f1: 0.6954216
Micro-average prec: 0.69769293, rec: 0.6894654, f1: 0.69355476
Quality on test dataset: 
time to finish evaluation: 1.60s
Total test loss: 45.4070	Avg test loss: 3.2434
label tp fp fn prec rec f1
I-Disease 494 277 295 0.6407263 0.626109 0.6333333
B-Disease 455 176 253 0.7210777 0.6426554 0.6796117
tp: 949 fp: 453 fn: 548 labels: 2
Macro-average prec: 0.680902, rec: 0.6343822, f1: 0.65681934
Micro-average prec: 0.67689013, rec: 0.63393456, f1: 0.65470856


Epoch 3/20 started, lr: 9.90099E-4, dataset size: 2589


Epoch 3/20 - 12.14s - loss: 158.39478 - avg training loss: 3.7713041 - batches: 42
Quality on validation dataset (20.0%), validation examples = 517
time to finish evaluation: 1.69s
Total validation loss: 31.2871	Avg validation loss: 2.4067
label tp fp fn prec rec f1
I-Disease 438 156 206 0.7373737 0.6801242 0.7075929
B-Disease 440 111 188 0.7985481 0.7006369 0.74639523
tp: 878 fp: 267 fn: 394 labels: 2
Macro-average prec: 0.7679609, rec: 0.6903806, f1: 0.72710717
Micro-average prec: 0.7668122, rec: 0.6902516, f1: 0.7265205
Quality on test dataset: 
time to finish evaluation: 1.65s
Total test loss: 39.7890	Avg test loss: 2.8421
label tp fp fn prec rec f1
I-Disease 478 161 311 0.74804384 0.6058302 0.66946787
B-Disease 475 144 233 0.7673667 0.670904 0.71590054
tp: 953 fp: 305 fn: 544 labels: 2
Macro-average prec: 0.7577053, rec: 0.63836706, f1: 0.6929355
Micro-average prec: 0.75755167, rec: 0.6366066, f1: 0.691833


Epoch 4/20 started, lr: 9.852217E-4, dataset size: 2589


Epoch 4/20 - 11.40s - loss: 113.478294 - avg training loss: 2.7018642 - batches: 42
Quality on validation dataset (20.0%), validation examples = 517
time to finish evaluation: 1.54s
Total validation loss: 25.3964	Avg validation loss: 1.9536
label tp fp fn prec rec f1
I-Disease 441 94 203 0.82429904 0.6847826 0.74809164
B-Disease 463 108 165 0.81085813 0.7372612 0.77231026
tp: 904 fp: 202 fn: 368 labels: 2
Macro-average prec: 0.81757855, rec: 0.7110219, f1: 0.76058626
Micro-average prec: 0.81735986, rec: 0.7106918, f1: 0.7603028
Quality on test dataset: 
time to finish evaluation: 1.61s
Total test loss: 34.0554	Avg test loss: 2.4325
label tp fp fn prec rec f1
I-Disease 477 96 312 0.83246076 0.60456276 0.7004406
B-Disease 506 142 202 0.7808642 0.71

As you see above, our earlyStopping feature worked, trainining was terminated before 30th epoch.

### Evaluate your model

In [0]:
pred_df = ner_model.stages[-1].transform(test_data)

In [0]:
pred_df.columns

Out[120]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|544.0|156.0|160.0|704.0| 0.7771|0.7727|0.7749|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
 macro|
+------------------+
0.7749287749287749|
+------------------+

None
+------------------+
 micro|
+------------------+
0.7749287749287749|
+------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+------+----+-----+------+---------+------+------+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+------+----+-----+------+---------+------+------+
Disease|1242.0|96.0|255.0|1497.0| 0.9283|0.8297|0.8762|
+-------+------+----+-----+------+---------+------+------+

+------------------+
 macro|
+------------------+
0.8761904761904762|
+------------------+

None
+------------------+
 micro|
+------------------+
0.8761904761904762|
+------------------+

None

In [0]:
ner_model.stages[1].write().overwrite().save('dbfs:/databricks/driver/models/custom_Med_NER_20e')

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = MedicalNerModel.load("dbfs:/databricks/driver/models/custom_Med_NER_20e")\
 .setInputCols(["sentence", "token", "embeddings"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "She has a metastatic breast cancer to lung"

def get_preds(text, light_model):

    result = light_model.fullAnnotate(text)[0]

    return [(i.result, i.metadata['entity']) for i in result['ner_span']]

get_preds(text, light_model)

Out[125]: [('metastatic breast cancer', 'Disease'), ('lung', 'Disease')]

End of Notebook #